# Getting Started With RDDs

일단 먼저 sc를 입력하여 Pyspark 가 init 되어 있는지 확인합니다.

sc를 입력했을때, Pyspark 뭐시기가 나오면 됩니다, 근데 안된다면 밑에 두번째있는 주석을 빼고 실행해주시면 됩니다

In [3]:
sc

In [1]:
#from pyspark import SparkContext

In [1]:
#sc = SparkContext()

그러면 Read.txt 파일을 만들어서 읽어보겠습니다

Read.txt 파일 안에는 인디안 송의 일부가 들어있습니다.

# 인디안 송

One little

two little

three little indians

for little

five little

six little indians

seven little

eight little

nine little indians

ten little indians

END!

# TXT File 만들기

%%writefile 을 사용하여 Read.txt 파일을 만들고, cat을 사용하여 파일을 읽어보겠습니다.

In [4]:
%%writefile Read.txt
One little
two little
three little indians
for little
five little
six little indians
seven little
eight little
nine little indians
ten little indians
End

Writing Read.txt


In [5]:
cat Read.txt

One little
two little
three little indians
for little
five little
six little indians
seven little
eight little
nine little indians
ten little indians
End

잘 읽혀 졌으면, 이제 RDD를 만들고, 택스트파일에서 indians 가 들어간 문장에서 첫번째인 것을 뽑아보겠습니다.

textFile() method 를 사용하여 읽고, filter() mehod 를 사용해서 필터링을 해주면 됩니다

그리고 .first() method 를 사용해서 첫번째인 문장을 보여주면 됩니다

In [6]:
lines = sc.textFile("Read.txt") # Read.txt 를 읽고,
pythonLines = lines.filter(lambda line: "indians" in line)
pythonLines.first()

u'three little indians'

# Read It from Memory

In practice, you will often use persist to load a subset of your data into memory and

query it repeatedly. For example, if we knew that we wanted to compute multiple results

about the README lines that contain “indians”, we could write

근데 이걸 해본 결과 에러가 납니다 :'( 이유는 모르겠네요 ㅠㅠ 

In [7]:
pythonLines.count()

4

In [8]:
pythonLines.first()

u'three little indians'

# 정리를 하자면...

1. 외부 데이터에서 RDD 를 생성한다

2. filter() 함수 같은걸 사용해서, transformations 를 한다

3. Spark 한테 persist 함수를 사용해서 재사용할것인지 물어본다

4. count() 나 first() 함수 같은것을 사용하여, Action 을 취해준다 

# RDD 를 생성해봅시다!

RDD 를 생성하려면, parallelize method 혹은, textFile method 를 사용해서 생성합니다

마치...

lines = sc.parallelize(["pandas", "i like pandas"]) 같이 써도 되고

liens = sc.textFile("어이/거기/README.md/위치/알려주이소") 같이 써도 됩니다

# Transformations Code...

Transforamations 는 RDD의 오퍼레이션 으로써, 새로운 RDD 를 만들어 주는 역할을 합니다

이것(Transformations)은 데이터의 이합집산, 지지고 볶는 흐름을 이야기합니다

Actions 는 '자 모든 Transformations 된 결과를 내놓아라~' 하는 명령어 입니다

trans → trans → trans → ... → action 같은 구성이 됩니다

그러면 여기서 예시로 Log.txt 에서 Error 을 필터링 해보도록 하겠습니다.

우리는 가짜의 log.txt 파일을 만들어 보겠습니다

In [9]:
%%writefile log.txt
error -- Connect to Google.com -- 401
pass -- Connect to Googler.com -- 200
error -- Download File From Apache.com -- 200
error -- Connect to Naver.com -- 404
pass -- Download File From Jeasus.com -- 200
pass -- Connect to AntiVerichip.com -- 404

Writing log.txt


In [10]:
cat log.txt

error -- Connect to Google.com -- 401
pass -- Connect to Googler.com -- 200
error -- Download File From Apache.com -- 200
error -- Connect to Naver.com -- 404
pass -- Download File From Jeasus.com -- 200
pass -- Connect to AntiVerichip.com -- 404

In [11]:
inputRDD = sc.textFile("log.txt")
errorsRDD = inputRDD.filter(lambda x: "error" in x)
errorsRDD.first()

u'error -- Connect to Google.com -- 401'

우리는 로그 택스트 파일을 읽어서 inputRDD 파일을 읽어보았습니다.

그러면 union 함수를 사용해서 badLinesRDD 함수를 하나 만든 후에, 거기에 errorsRDD 와 warningsRDD 2개를 badLinesRDD 로 Union 시켜보도록 하겠습니다

이때 union 함수를 사용하게 됩니다

![유니온함수](http://www.yongbi.net/attach/1/1389250610.png)

그러면 저기 로그 파일을 살짝! 바꿔서 Warning 을 만들고, Union을 시켜보도록 하겠습니다

In [12]:
%%writefile log2.txt
error -- Connect to Google.com -- 401
pass -- Connect to Googler.com -- 200
warning -- Try Download File from SomeVideos.com -- 451
pass -- Try Download File From Apache.com -- 200
error -- Connect to Naver.com -- 404
pass -- Try Download File From Jeasus.com -- 200
pass -- Connect to AAAJESUS.com -- 404
warning -- Connect to Somewhere.com -- 451 

Writing log2.txt


In [13]:
cat log2.txt

error -- Connect to Google.com -- 401
pass -- Connect to Googler.com -- 200
warning -- Try Download File from SomeVideos.com -- 451
pass -- Try Download File From Apache.com -- 200
error -- Connect to Naver.com -- 404
pass -- Try Download File From Jeasus.com -- 200
pass -- Connect to AntiVerichip.com -- 404
warning -- Connect to Uporn.com -- 451 

In [15]:
indputRDD = sc.textFile("log2.txt")
errorsRDD = inputRDD.filter(lambda x: "error" in x)
warningsRDD = inputRDD.filter(lambda x: "warning" in x)
badLinesRDD = errorsRDD.union(warningsRDD)
print badLinesRDD.count()
for line in badLinesRDD.take(10):
    print line

3
error -- Connect to Google.com -- 401
error -- Download File From Apache.com -- 200
error -- Connect to Naver.com -- 404


# Passing Functions to Spark

## 파이썬으로 함수를 넘기는 방법
- 람다 표기법을 이용하는 방식
- 모듈 상의 탑레벨 함수(Top-level function)
- 파이썬 클래스 안에서 함수를 정의하여, 클래스 안에서 그 함수를 스파크로 넘기는 방식

## 1. 람다 표기법을 이용하는 방식
     word = rdd.filter(lambda s: "error" in s)
     
## 2. 모듈 상의 탑레벨 함수를 사용하는 방법
     def containsError(s):
     return "error" in s
     word = rdd.filter(containsError)
     
## 3. 파이썬 클래스 안에서 함수를 정의하여, 클래스 안에서 그 함수를 스파크로 넘기는 방식
     class SearchFunctions(object):
        def __init__(self, query):
           self.query = query
     def isMatch(self, s):
        return query in s
     def getMatchesFunctionReference(self, rdd):
        # Problem: references all of "self" in "self.isMatch"
        return rdd.filter(self.isMatch)
     def getMatchesMemberReference(self, rdd):
        # Problem: references all of "self" in "self.query"
        return rdd.filter(lambda x: self.query in x)

문제는 객체 전체가 스파크로 넘어가게 되므로, 메모리를 과 사용할수 있게 된다 그래서 이런 문제를 피하기 위해서는...

     class WordFunctions(object):
     ...
     def getMatchesNoReference(self, rdd):
         # Safe: extract only the field we need into a local variable
         query = self.query
         return rdd.filter(lambda x: query in x)

와 같은 코드를 사용하여 로컬 변수 x  의 값만 넘겨주어 메모리의 과 사용을 방지할수 있다

# Some Transformations

BASIC RDD Transformations on an RDD Containing {1, 2, 3, 3}

Function Name | 설명
------------- | ------------- 
map(func) | 새로운 Distributed Dataset 을 선택된 값들 중에서 리턴해준다 
filter(func) | 람다식에 조건에 따라서 참인 결과값을 리턴해준다 
distinct(func) | 중복되는 값이 있으면 한 번만 나오게 해 준다 
flatmap(func) | flatMap 은 map 함수와 비슷하지만, 들어간 값이 다 나오지 않는다

# 1.  Map()
- Map() 함수를 집어넣었을때 하나가 들어가면 다 나온다.

In [16]:
nums = sc.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x).collect()
for num in squared:
    print "%i " % (num)

1 
4 
9 
16 


# 2. Flatmap Example
- Map() 함수와 같지만, 모든 값을 집어넣었을때 값의 갯수가 다르다

In [17]:
lines = sc.parallelize(["hello world", "hi"])
words = lines.flatMap(lambda line: line.split(" "))
words.first() # returns "hello"

'hello'

Two-RDD transformations on RDDs containing {1, 2, 3} and {3, 4, 5}

Fiunction Name|Purpose|Example|Result
------------|-----------
Union|RDD A 와 RDD B 의 합집합|rdd.union(other)|{1, 2, 3, 3, 4, 5}
Intersection|RDD A와 RDD B 의 교집합|rdd.intersection(other)|{3}
Subtract|RDD A에 있는것들중에서 RDD B 에 공통되어 있는것들을 삭제|rdd.substract(other)|{1, 2}
Cartesian|각각에 있는것들중에서 합치기|rdd.cartesian(other)|{(1, 3), (1, 4), (3,5)}

# Actions

- Reduce 함수 는 RDD 값들을 병렬로 병합연산해주는 역할을 한다, 같은 타입을 리턴해준다

      sum = rdd.reduce(lambda x, y: x + y)
     

- Aggregate 함수 는 Reduce 함수와 유사하나 다른 타입을 리턴해준다.

      sumCount = nums.aggregate((0, 0),
        (lambda x, y: (x[0] + y, x[1] + 1),
        (lambda x, y: (x[0] + y[0], x[1] + y[1]))))
      return sumCount[0] / float(sumCount[1])
      
그 외의 많은 함수는 [여기](http://pubdata.tistory.com/38)서 읽어볼수 있습니다